# Importing Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/Machine learning

/content/drive/My Drive/Machine learning


In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
import xgboost as xgb

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score

pd.set_option("display.max_columns", 100)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier 

In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

In [ ]:
!pip install catboost

     |████████████████████████████████| 64.8MB 64kB/s 


In [ ]:
from catboost import CatBoostClassifier,Pool

In [ ]:
#methods for distance calculation
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a)) 

In [ ]:
features_df = pd.read_csv(
    "train.csv", 
    index_col="tripid"
)


# Preprocessing data

In [ ]:
#removing noisy locations
features_df.drop( features_df[ features_df['pick_lat'] < 5.9 ].index , inplace=True)
features_df.drop( features_df[ features_df['pick_lat'] > 9.82 ].index , inplace=True)	
features_df.drop( features_df[ features_df['pick_lon'] < 79 ].index , inplace=True)
features_df.drop( features_df[ features_df['pick_lon'] > 82 ].index , inplace=True)
features_df.drop( features_df[ features_df['drop_lat'] < 5.9 ].index , inplace=True)
features_df.drop( features_df[ features_df['drop_lat'] > 9.82 ].index , inplace=True)	
features_df.drop( features_df[ features_df['drop_lon'] < 79 ].index , inplace=True)
features_df.drop( features_df[ features_df['drop_lon'] > 82 ].index , inplace=True)

In [ ]:
#removing dupicates
features_df = features_df.drop_duplicates()

In [ ]:
features_df

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,correct
213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,correct
213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,correct


In [ ]:
features_df["distance"] =  distance(features_df["pick_lat"],features_df["pick_lon"],features_df["drop_lat"],features_df["drop_lon"])

In [ ]:
features_df["pickup_time"] = pd.to_datetime(features_df["pickup_time"],errors = "coerce")
features_df["drop_time"] = pd.to_datetime(features_df["drop_time"],errors = "coerce")
features_df['duration'] = features_df['duration'].fillna((features_df['drop_time'] - features_df['pickup_time']).astype('timedelta64[s]'))

# Nan Estimation

In [ ]:
features_df['additional_fare'] = features_df['additional_fare'].fillna(features_df['additional_fare'].mode().iloc[0])

In [ ]:
features_df_correct = features_df[features_df.label=='correct']
features_df_incorrect = features_df[features_df.label=='incorrect']

In [ ]:
features_df_correct['meter_waiting'] = features_df_correct['meter_waiting'].fillna(features_df_correct['meter_waiting'].mean())
features_df_incorrect['meter_waiting'] = features_df_incorrect['meter_waiting'].fillna(features_df_incorrect['meter_waiting'].mean())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
features_df_correct['meter_waiting_fare'] = features_df_correct['meter_waiting_fare'].fillna(features_df_correct['meter_waiting_fare'].mean())
features_df_incorrect['meter_waiting_fare'] = features_df_incorrect['meter_waiting_fare'].fillna(features_df_incorrect['meter_waiting_fare'].mean())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# fare nans replacement with considering class means
features_df_correct['fare'] = features_df_correct['fare'].fillna((92.33)*features_df_correct['distance']+features_df_correct['additional_fare']+features_df_correct['meter_waiting_fare'])
features_df_incorrect['fare'] = features_df_incorrect['fare'].fillna((221)*features_df_incorrect['distance']+features_df_incorrect['additional_fare']+features_df_incorrect['meter_waiting_fare'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# features_df = pd.concat([features_df_correct,features_df_incorrect],sort=True)

In [ ]:
# features_df.describe()

In [ ]:
y= features_df["label"]

In [ ]:
y =y.replace(to_replace="correct",value=1)
y =y.replace(to_replace="incorrect",value=0)

In [ ]:
X = features_df.drop(columns=["label"], axis=1)

In [ ]:
test_features_df = pd.read_csv("test.csv", 
                               index_col="tripid")

In [ ]:
test_features_df

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
tripid,,,,,,,,,,,,
213284604,10.5,924,42,2.44860,148,2/1/2020 0:38,2/1/2020 0:53,6.83454,79.8750,6.77490,79.8840,289.27
213286352,10.5,4249,20,0.00000,91,2/1/2020 1:02,2/1/2020 2:13,6.91168,79.8723,6.55091,79.9706,1912.70
213293973,10.5,1552,255,2.65880,23,2/1/2020 5:02,2/1/2020 5:28,6.92145,79.8478,6.90539,79.8989,394.00
213294622,10.5,462,16,0.00000,198,2/1/2020 5:30,2/1/2020 5:38,6.77433,79.9416,6.80401,79.9407,154.32
213298687,10.5,814,392,12.36920,69,2/1/2020 7:00,2/1/2020 7:14,6.97968,79.9130,6.98875,79.8914,147.47
...,...,...,...,...,...,...,...,...,...,...,...,...
222856243,10.5,1723,429,24.83332,3,3/16/2020 21:28,3/16/2020 21:56,6.85103,79.9567,6.85588,79.9214,388.48
222857785,10.5,1378,80,0.00000,125,3/16/2020 21:59,3/16/2020 22:22,6.91293,79.9656,6.92112,79.8980,379.85
222858416,10.5,418,56,3.28440,93,3/16/2020 22:02,3/16/2020 22:09,6.85718,79.9081,6.83868,79.9083,112.79


In [ ]:
X

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,distance
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,2019-11-01 00:20:00,2019-11-01 00:34:00,6.86252,79.8993,6.90330,79.8783,270.32,3.164501
189125358,10.5,791.0,47.0,0.0000,134.0,2019-11-01 00:56:00,2019-11-01 01:09:00,6.88589,79.8984,6.91373,79.8923,197.85,1.968540
189125719,10.5,1087.0,80.0,0.0000,61.0,2019-11-01 01:08:00,2019-11-01 01:26:00,6.90839,79.8651,6.93669,79.9146,301.64,3.917991
189127273,10.5,598.0,271.0,15.6638,68.0,2019-11-01 02:27:00,2019-11-01 02:37:00,6.92570,79.8895,6.92748,79.8971,82.30,0.535588
189128020,10.5,1020.0,NaN,NaN,NaN,2019-11-01 03:34:00,2019-11-01 03:51:00,6.87441,79.8615,6.84478,79.9290,358.39,5.062797
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213803193,10.5,838.0,93.0,5.4219,451.0,2020-01-31 22:07:00,2020-01-31 22:21:00,7.29073,80.6367,7.28891,80.6557,198.26,1.308220
213812756,10.5,2151.0,428.0,0.0000,39.0,2020-01-31 23:07:00,2020-01-31 23:43:00,6.90569,79.8516,6.95089,79.9389,581.23,6.753297
213813930,10.5,263.0,9.0,0.0000,110.0,2020-01-31 23:21:00,2020-01-31 23:25:00,7.09210,79.9000,7.10135,79.9017,76.20,0.649655


In [ ]:
#X = X.dropna()

In [ ]:
X

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,distance
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,2019-11-01 00:20:00,2019-11-01 00:34:00,6.86252,79.8993,6.90330,79.8783,270.32,3.164501
189125358,10.5,791.0,47.0,0.0000,134.0,2019-11-01 00:56:00,2019-11-01 01:09:00,6.88589,79.8984,6.91373,79.8923,197.85,1.968540
189125719,10.5,1087.0,80.0,0.0000,61.0,2019-11-01 01:08:00,2019-11-01 01:26:00,6.90839,79.8651,6.93669,79.9146,301.64,3.917991
189127273,10.5,598.0,271.0,15.6638,68.0,2019-11-01 02:27:00,2019-11-01 02:37:00,6.92570,79.8895,6.92748,79.8971,82.30,0.535588
189128020,10.5,1020.0,NaN,NaN,NaN,2019-11-01 03:34:00,2019-11-01 03:51:00,6.87441,79.8615,6.84478,79.9290,358.39,5.062797
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213803193,10.5,838.0,93.0,5.4219,451.0,2020-01-31 22:07:00,2020-01-31 22:21:00,7.29073,80.6367,7.28891,80.6557,198.26,1.308220
213812756,10.5,2151.0,428.0,0.0000,39.0,2020-01-31 23:07:00,2020-01-31 23:43:00,6.90569,79.8516,6.95089,79.9389,581.23,6.753297
213813930,10.5,263.0,9.0,0.0000,110.0,2020-01-31 23:21:00,2020-01-31 23:25:00,7.09210,79.9000,7.10135,79.9017,76.20,0.649655


In [ ]:
test_features_df = test_features_df.fillna(test_features_df.mean())

# Feature Generation

In [ ]:
X["pickup_time"] = pd.to_datetime(X["pickup_time"],errors = "coerce")

In [ ]:
X["drop_time"] = pd.to_datetime(X["drop_time"],errors = "coerce")

In [ ]:
X["pickup_time_hour"] = X["pickup_time"].dt.hour

In [ ]:
X["pickup_time_minute"] = X["pickup_time"].dt.minute

In [ ]:
X["drop_time_hour"] =X["drop_time"].dt.hour

In [ ]:
X["drop_time_minute"] =X["drop_time"].dt.minute

In [ ]:
X["pickup_time_day"] = X["pickup_time"].dt.day
# X["pickup_time_month"] = X["pickup_time"].dt.month
X["drop_time_day"] = X["drop_time"].dt.day
# X["drop_time_month"] = X["drop_time"].dt.month
# X["pick_up_year"] = X["drop_time"].dt.year
# X["drop_year"] = X["drop_time"].dt.year

In [ ]:
X.dtypes

additional_fare                     float64
duration                            float64
meter_waiting                       float64
meter_waiting_fare                  float64
meter_waiting_till_pickup           float64
pickup_time                  datetime64[ns]
drop_time                    datetime64[ns]
pick_lat                            float64
pick_lon                            float64
drop_lat                            float64
drop_lon                            float64
fare                                float64
distance                            float64
pickup_time_hour                      int64
pickup_time_minute                    int64
drop_time_hour                        int64
drop_time_minute                      int64
pickup_time_day                       int64
drop_time_day                         int64
dtype: object

In [ ]:
X["effective_time"] = X["duration"]-X["meter_waiting"]


In [ ]:
X.loc[X['fare'] <= 2000, 'fare_outlier'] = 1
X.loc[X['fare'] > 2000, 'fare_outlier'] = 0

In [ ]:
X['meter_waiting_fare_diff'] = X['meter_waiting_fare'] - X['meter_waiting']*0.057

In [ ]:
X['fare_mean'] = (X['fare'] - X['meter_waiting_fare'])/(X['duration'] - X['meter_waiting'])

In [ ]:
# X.loc[X['additional_fare'] <= 100, 'additional_fare_outlier'] = 1
# X.loc[X['additional_fare'] > 100, 'additional_fare_outlier'] = 0

In [ ]:
# X.loc[X['meter_waiting'] <= 2000, 'meter_waiting_outlier'] = 1
# X.loc[X['meter_waiting'] > 2000, 'meter_waiting_outlier'] = 0

In [ ]:
# X.loc[X['duration'] <= 6000, 'duration_outlier'] = 1
# X.loc[X['duration'] > 6000, 'duration_outlier'] = 0

In [ ]:
# X.loc[X['meter_waiting_fare'] <= 114, 'meter_waiting_fare_outlier'] = 1
# X.loc[X['meter_waiting_fare'] > 114, 'meter_waiting_fare_outlier'] = 0

In [ ]:
X['speed'] = X['distance']/X['effective_time']

In [ ]:
X['effective_fare'] = X['fare'] - X['meter_waiting_fare'] 

In [ ]:
# X['duration'] = X['duration'].fillna((X['drop_time'] - X['pickup_time']).astype('timedelta64[s]'))
X['Correct_fare'] = 92.33*X['distance']+0.057+X['meter_waiting']*0.057

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
test_features_df["pickup_time"] = pd.to_datetime(test_features_df["pickup_time"],errors = "coerce")
test_features_df["drop_time"] = pd.to_datetime(test_features_df["drop_time"],errors = "coerce")
test_features_df["distance"] =  distance(test_features_df["pick_lat"],test_features_df["pick_lon"],test_features_df["drop_lat"],test_features_df["drop_lon"])
test_features_df["pickup_time_hour"] = test_features_df["pickup_time"].dt.hour
test_features_df["pickup_time_minute"] = test_features_df["pickup_time"].dt.minute
test_features_df["drop_time_hour"] =test_features_df["drop_time"].dt.hour
test_features_df["drop_time_minute"] =test_features_df["drop_time"].dt.minute
test_features_df["effective_time"] = test_features_df["duration"]-test_features_df["meter_waiting"]
test_features_df["pickup_time_day"] = test_features_df["pickup_time"].dt.day
# test_features_df["pickup_time_month"] = test_features_df["pickup_time"].dt.month
test_features_df["drop_time_day"] = test_features_df["drop_time"].dt.day
# test_features_df["drop_time_month"] = test_features_df["drop_time"].dt.month
# test_features_df["pick_up_year"] = test_features_df["drop_time"].dt.year
# test_features_df["drop_year"] = test_features_df["drop_time"].dt.year
# test_features_df["distance"] =(test_features_df["pick_lat"]-test_features_df["drop_lat"])**2 + (test_features_df["pick_lon"]-test_features_df["drop_lon"])**2



In [ ]:
# test_features_df["effective_time"] = (test_features_df["effective_time"] -test_features_df["effective_time"].mean() )/test_features_df["effective_time"].std() 
# test_features_df["effective_fare"] = (test_features_df["effective_fare"] -test_features_df["effective_fare"].mean() )/test_features_df["effective_fare"].std() 
# test_features_df["mean_fare_for_unit_length"] = (test_features_df["mean_fare_for_unit_length"] -test_features_df["mean_fare_for_unit_length"].mean() )/test_features_df["mean_fare_for_unit_length"].std() 
# test_features_df["duration_fare"] = (test_features_df["duration_fare"] -test_features_df["duration_fare"].mean() )/test_features_df["duration_fare"].std() 


In [ ]:
# test_features_df = test_features_df.drop(columns=["pickup_time","drop_time"],axis=1)
test_features_df.loc[test_features_df['fare'] <= 2000, 'fare_outlier'] = 1
test_features_df.loc[test_features_df['fare'] > 2000, 'fare_outlier'] = 0

In [ ]:
test_features_df['meter_waiting_fare_diff'] = test_features_df['meter_waiting_fare'] - test_features_df['meter_waiting']*0.057

In [ ]:
test_features_df['fare_mean'] = (test_features_df['fare'] - test_features_df['meter_waiting_fare'])/(test_features_df['duration'] - test_features_df['meter_waiting'])

In [ ]:
test_features_df['speed'] = test_features_df['distance']/test_features_df['effective_time']

In [ ]:
test_features_df['effective_fare'] = test_features_df['fare'] - test_features_df['meter_waiting_fare'] 

In [ ]:
# test_features_df.loc[test_features_df['additional_fare'] <= 100, 'additional_fare_outlier'] = 1
# test_features_df.loc[test_features_df['additional_fare'] > 100, 'additional_fare_outlier'] = 0
test_features_df['Correct_fare'] = 92.33*test_features_df['distance']+10.5+test_features_df['meter_waiting']*0.057

In [ ]:
# test_features_df.loc[test_features_df['meter_waiting'] <= 2000, 'meter_waiting_outlier'] = 1
# test_features_df.loc[test_features_df['meter_waiting'] > 2000, 'meter_waiting_outlier'] = 0

In [ ]:
# test_features_df.loc[test_features_df['duration'] <= 6000, 'duration_outlier'] = 1
# test_features_df.loc[test_features_df['duration'] > 6000, 'duration_outlier'] = 0

In [ ]:
# test_features_df.loc[test_features_df['meter_waiting_fare'] <= 114, 'meter_waiting_fare_outlier'] = 1
# test_features_df.loc[test_features_df['meter_waiting_fare'] > 114, 'meter_waiting_fare_outlier'] = 0

In [ ]:
# test_features_df['speed'] = test_features_df['distance']/test_features_df['effective_time']

In [ ]:
categorical_var1 = np.where(test_features_df.dtypes != np.float )[0]

In [ ]:
categorical_var1

array([ 1,  2,  4,  5,  6, 13, 14, 15, 16, 17, 18, 19])

In [ ]:
test_features_df.dtypes

additional_fare                     float64
duration                              int64
meter_waiting                         int64
meter_waiting_fare                  float64
meter_waiting_till_pickup             int64
pickup_time                  datetime64[ns]
drop_time                    datetime64[ns]
pick_lat                            float64
pick_lon                            float64
drop_lat                            float64
drop_lon                            float64
fare                                float64
distance                            float64
pickup_time_hour                      int64
pickup_time_minute                    int64
drop_time_hour                        int64
drop_time_minute                      int64
effective_time                        int64
pickup_time_day                       int64
drop_time_day                         int64
fare_outlier                        float64
meter_waiting_fare_diff             float64
fare_mean                       

In [ ]:
X.dtypes

additional_fare                     float64
duration                            float64
meter_waiting                       float64
meter_waiting_fare                  float64
meter_waiting_till_pickup           float64
pickup_time                  datetime64[ns]
drop_time                    datetime64[ns]
pick_lat                            float64
pick_lon                            float64
drop_lat                            float64
drop_lon                            float64
fare                                float64
distance                            float64
pickup_time_hour                      int64
pickup_time_minute                    int64
drop_time_hour                        int64
drop_time_minute                      int64
pickup_time_day                       int64
drop_time_day                         int64
effective_time                      float64
fare_outlier                        float64
meter_waiting_fare_diff             float64
fare_mean                       

In [ ]:
categorical_var = np.where(X.dtypes != np.float )[0]

In [ ]:
categorical_var

array([ 5,  6, 13, 14, 15, 16, 17, 18])

In [ ]:
eval_dataset = Pool(X_test,y_test)

In [ ]:
eval_dataset

# Model training

In [ ]:
model = CatBoostClassifier(iterations=6000,nan_mode='Max')
# model = CatBoostClassifier(eval_metric='F1')

In [ ]:
model.fit(X,y,plot=True)

# model.fit(X_train,y_train,eval_set=eval_dataset,verbose=False)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Streaming output truncated to the last 5000 lines.
1001:	learn: 0.1165329	total: 17.1s	remaining: 1m 25s
1002:	learn: 0.1165175	total: 17.2s	remaining: 1m 25s
1003:	learn: 0.1164924	total: 17.2s	remaining: 1m 25s
1004:	learn: 0.1164633	total: 17.2s	remaining: 1m 25s
1005:	learn: 0.1164349	total: 17.2s	remaining: 1m 25s
1006:	learn: 0.1164150	total: 17.2s	remaining: 1m 25s
1007:	learn: 0.1163941	total: 17.2s	remaining: 1m 25s
1008:	learn: 0.1163776	total: 17.3s	remaining: 1m 25s
1009:	learn: 0.1163468	total: 17.3s	remaining: 1m 25s
1010:	learn: 0.1163328	total: 17.4s	remaining: 1m 25s
1011:	learn: 0.1163018	total: 17.4s	remaining: 1m 25s
1012:	learn: 0.1162761	total: 17.4s	remaining: 1m 25s
1013:	learn: 0.1162447	total: 17.4s	remaining: 1m 25s
1014:	learn: 0.1162212	total: 17.4s	remaining: 1m 25s
1015:	learn: 0.1162081	total: 17.4s	remaining: 1m 25s
1016:	learn: 0.1161928	total: 17.5s	remaining: 1m 25s
1017:	learn: 0.1161710	total: 17.5s	remaining: 1m 25s
1018:	learn: 0.1161553	total: 1

In [ ]:
# print(model.get_best_iteration())

In [ ]:
prediction = model.predict(test_features_df)

In [ ]:
prediction

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
submission_df = pd.read_csv("sample_submission.csv", 
                            index_col="tripid")

In [ ]:
X.head()


,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,distance,pickup_time_hour,pickup_time_minute,drop_time_hour,drop_time_minute,pickup_time_day,drop_time_day,effective_time,fare_outlier,meter_waiting_fare_diff,fare_mean,speed,effective_fare,Correct_fare
tripid,,,,,,,,,,,,,,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,2019-11-01 00:20:00,2019-11-01 00:34:00,6.86252,79.8993,6.90330,79.8783,270.32,3.164501,0,20,0,34,1,1,778.0,1.0,-3.1920,0.347455,0.004067,270.3200,295.427336
189125358,10.5,791.0,47.0,0.0000,134.0,2019-11-01 00:56:00,2019-11-01 01:09:00,6.88589,79.8984,6.91373,79.8923,197.85,1.968540,0,56,1,9,1,1,744.0,1.0,-2.6790,0.265927,0.002646,197.8500,184.491274
189125719,10.5,1087.0,80.0,0.0000,61.0,2019-11-01 01:08:00,2019-11-01 01:26:00,6.90839,79.8651,6.93669,79.9146,301.64,3.917991,1,8,1,26,1,1,1007.0,1.0,-4.5600,0.299543,0.003891,301.6400,366.365101
189127273,10.5,598.0,271.0,15.6638,68.0,2019-11-01 02:27:00,2019-11-01 02:37:00,6.92570,79.8895,6.92748,79.8971,82.30,0.535588,2,27,2,37,1,1,327.0,1.0,0.2168,0.203780,0.001638,66.6362,64.954886
189128020,10.5,1020.0,NaN,NaN,NaN,2019-11-01 03:34:00,2019-11-01 03:51:00,6.87441,79.8615,6.84478,79.9290,358.39,5.062797,3,34,3,51,1,1,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
np.testing.assert_array_equal(test_features_df.index.values, 
                              submission_df.index.values)


In [ ]:
submission_df["prediction"] = prediction[:]

In [ ]:
submission_df.head()

,prediction
tripid,
213284604,1
213286352,1
213293973,1
213294622,1
213298687,1


In [ ]:
submission_df.to_csv('Pro_Ultra_Cosmic_Catestropic_Multi_Higher_dimensional_Gamma_Mission.csv', index=True)

In [ ]:
model.get_feature_importance()

array([ 2.26960927,  2.67133898,  2.38052946,  1.5004143 ,  7.43057231,
        2.88191395,  2.48406048,  2.86625919,  2.53423261,  1.74790353,
        2.12522772,  8.70980719,  4.29210981,  2.09221746,  2.19087032,
        1.60169772,  2.65083674,  1.21067067,  1.12946411,  2.42383337,
        0.16462252,  4.05091037, 23.30847916,  7.21315458,  4.67402618,
        3.395238  ])

In [ ]:
X.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,distance,pickup_time_hour,pickup_time_minute,drop_time_hour,drop_time_minute,pickup_time_day,drop_time_day,effective_time,fare_outlier,meter_waiting_fare_diff,fare_mean,speed,effective_fare,Correct_fare
tripid,,,,,,,,,,,,,,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,2019-11-01 00:20:00,2019-11-01 00:34:00,6.86252,79.8993,6.90330,79.8783,270.32,3.164501,0,20,0,34,1,1,778.0,1.0,-3.1920,0.347455,0.004067,270.3200,295.427336
189125358,10.5,791.0,47.0,0.0000,134.0,2019-11-01 00:56:00,2019-11-01 01:09:00,6.88589,79.8984,6.91373,79.8923,197.85,1.968540,0,56,1,9,1,1,744.0,1.0,-2.6790,0.265927,0.002646,197.8500,184.491274
189125719,10.5,1087.0,80.0,0.0000,61.0,2019-11-01 01:08:00,2019-11-01 01:26:00,6.90839,79.8651,6.93669,79.9146,301.64,3.917991,1,8,1,26,1,1,1007.0,1.0,-4.5600,0.299543,0.003891,301.6400,366.365101
189127273,10.5,598.0,271.0,15.6638,68.0,2019-11-01 02:27:00,2019-11-01 02:37:00,6.92570,79.8895,6.92748,79.8971,82.30,0.535588,2,27,2,37,1,1,327.0,1.0,0.2168,0.203780,0.001638,66.6362,64.954886
189128020,10.5,1020.0,NaN,NaN,NaN,2019-11-01 03:34:00,2019-11-01 03:51:00,6.87441,79.8615,6.84478,79.9290,358.39,5.062797,3,34,3,51,1,1,NaN,1.0,NaN,NaN,NaN,NaN,NaN
